In [18]:
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import time
from datetime import date
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/Users/dom/opt/anaconda3/envs/hotelstats/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
def geocode_address(address):
    if pd.isna(address):
        return None

    # Initialize the geocoder with the appropriate user_agent and user_agent_headers
    user_agent = 'Datachalet'
    geolocator = Nominatim(user_agent=user_agent)

    # Geocode the address to get the latitude and longitude
    location = geolocator.geocode(address)

    if location:
        latitude = location.latitude
        longitude = location.longitude

        # Pause for 1 second before the next geocoding request
        time.sleep(1.2)

        return latitude, longitude
    else:
        return None


In [6]:
# current_date = date.today().strftime("%Y%m%d")
# df = pd.read_feather(f"data/{current_date}_Scraped_Data_Hotels.feather")
# df['Address'] = df['Strasse'] + ', ' + df['PLZ'] + ', ' + df['Ort']
# # Assuming you have already loaded the DataFrame into the variable df
# # Apply the geocode_address function to the "Address" column
# df[['Latitude', 'Longitude']] = df['Address'].apply(geocode_address).apply(pd.Series)

In [8]:
# # Export the scraped Data
# current_date = date.today().strftime("%Y%m%d")
# filename = f"data/{current_date}_Scraped_Data_Hotels_Geo.feather"
# df.to_feather(filename)

In [79]:
df = pd.read_feather(f"data/20230718_Scraped_Data_Hotels_Geo.feather")
gemeinden_df = pd.read_excel('data/gemeindenbfs.xlsx', sheet_name='GDE')
plz_df = plz_df = pd.read_csv('data/plz_verzeichnis.csv',sep=';')
df_urls = pd.read_feather(f"data/20230718_Scraped_Data_Urls.feather")

In [80]:
# Convert 'PLZ' column to numeric type and replace non-numeric values with NaN
#df['PLZ'] = pd.to_numeric(df['PLZ'], errors='coerce')
merged_df = pd.merge(df,plz_df, left_on='Ort', right_on='ortbez18', how='left')
merged_df = pd.merge(merged_df,df_urls, left_on='url', right_on='Link', how='left')
merged_df = merged_df.drop_duplicates(subset='Hotel').reset_index()

In [82]:
merged_df.Stars.value_counts()

3-Sterne Hotel                  590
4-Sterne Hotel                  372
Swiss Lodge                     355
3-Sterne Superior Hotel         143
4-Sterne Superior Hotel         113
5-Sterne Superior Hotel          69
2-Sterne Hotel                   63
5-Sterne Hotel                   36
2-Sterne Superior Hotel          20
1-Stern Superior Hotel            7
1-Stern Hotel                     6
5-Sterne Serviced Apartments      3
4-Sterne Serviced Apartments      3
3-Sterne Serviced Apartments      2
2-Sterne Serviced Apartments      1
Name: Stars, dtype: int64

In [83]:
merged_df = merged_df.rename(columns={"Latitude": "lat", "Longitude": "lon"})

In [84]:
# Export the scraped Data
current_date = date.today().strftime("%Y%m%d")
filename = f"data/{current_date}_Hotels.feather"
merged_df.to_feather(filename)

In [85]:
df = pd.read_feather(f"data/20230721_Hotels.feather")

In [73]:
df.Anzahl_Zimmer_Apartments.info()

<class 'pandas.core.series.Series'>
RangeIndex: 3822 entries, 0 to 3821
Series name: Anzahl_Zimmer_Apartments
Non-Null Count  Dtype
--------------  -----
3685 non-null   Int64
dtypes: Int64(1)
memory usage: 33.7 KB
